In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# Image Visualization

In [ ]:
data_dir = "../input/bengaliai-cv19/"
files_train = [f'train_image_data_{fid}.parquet' for fid in range(4)]

In [ ]:
file_name = os.path.join(data_dir + files_train[0]) # 2개의 문자열을 결합하여 1개의 경로로 만들어주는 것

In [ ]:
train0 = pd.read_parquet(file_name)

In [ ]:
print(train0.shape) # 137 * 236 = 32332

In [ ]:
train0.head()

In [ ]:
idx = np.random.randint(len(train0))
img = train0.iloc[idx, 1:].values.astype(np.uint8) # 픽셀값이 0~255로 정해져있으므로 큰 데이터타입x

plt.imshow(255 - img.reshape(137, 236), cmap='gray')

# Mult-label stratification folding
어떤 데이터에 대해서 학습을 진행하고 어떤 데이터에 대해서 평가를 진행할지 결정하는 것은 매우 중요하다.
따라서 가지고 있는 데이터 셋을 train set과 validation set으로 나누어줘야 하는데, 한번만 나누는게 아니고 여러 번 나누게 되면 더 좋은 성능을 보여주게 된다.

In [ ]:
df_train = pd.read_csv(data_dir + "train.csv")

df_train.head()

In [ ]:
df_train.shape

In [ ]:
df_train['grapheme_root'].value_counts()

In [ ]:
# grapheme_root class의 분포를 출력
plt.figure(figsize=(10, 20))

df_train['grapheme_root'].value_counts().sort_index().plot.barh()

In [ ]:
# vowel_diacritic의 분포를 출력
plt.figure(figsize=(10, 20))

df_train['vowel_diacritic'].value_counts().sort_index().plot.barh()

In [ ]:
# consonant_diacritic	의 분포를 출력
plt.figure(figsize=(10, 20))

df_train['consonant_diacritic'].value_counts().sort_index().plot.barh()

class의 분포가 불균형함을 확인할 수 있다. 이 경우 랜덤하게 샘플링을 하게 되면 해당 분포가 제대로 반영되지 않을 수가 있다.
데이터가 편향되어 있을 경우 단순 k-겹 교차검증을 사용하면 성능 평가가 잘 되지 않을 수 있다. 

=> Stratified cross validation 사용

일반적으로 scikit-learn에 staratified folding이 존재하지만, class가 하나인 경우에만 사용이 가능하다. multi-class인 경우 iterative-stratification 라는 라이브러리를 사용하면 된다. 

In [ ]:
'Train_0'.split('_')

In [ ]:
df_train['id'] = df_train['image_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
df_train.head()

In [ ]:
X = df_train[['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values[:, 0] # id만 가져오는 것
y = df_train[['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values[:, 1:] # 나머지 모두를 가져오는 것

In [ ]:
!pip install iterative-stratification

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
mskf = MultilabelStratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [ ]:
df_train['fold'] = -1

df_train

In [ ]:
for i, (trn_idx, vid_idx) in enumerate(mskf.split(X, y)):
    df_train.loc[vid_idx, 'fold'] = i

In [ ]:
df_train['fold'].value_counts()

In [ ]:
df_train.to_csv("./df_folds.csv", index=False)

# Efficient learning process
데이터프레임을 row by row로 잘라서 학습 속도를 높이는 방법이 있다. 본 대회에서는 parquet 파일을 읽어서 4개의 파일을 합친 후 학습을 진행해야 하는데, 용량이 크기 때문에 메모리가 많지 않으면 학습하다가 다운되는 경우가 발생할 수도 있다. 또 pandas가 읽고 쓰는 속도가 느리기 때문에, pandas에서 데이터를 불러오는 식으로 짜게 되면 전체적인 학습시간이 길어지게 된다. 

pandas보다 numpy 배열로 불러오는 것이 훨씬 빠르다.

만약 램이 작다면 이미지 하나하나를 읽고 쓰는 방식으로 하는 것이 훨씬 빠르다.(피클이나 넘파이 형식으로 저장)

In [ ]:
import joblib # 디스크 캐싱, 병렬 프로그래밍하거나 학습한 모델을 저장할 수 있는 sklearn 라이브러리
from tqdm import tqdm # python 진행률 프로세스바

In [ ]:
for fname in files_train:
    F = os.path.join(data_dir, fname)
    df_train = pd.read_parquet(F)
    img_ids = df_train['image_id'].values
    img_array = df_train.iloc[:, 1:].values
    
    for idx in tqdm(range(len(df_train))):
        img_id = img_ids[idx]
        img = img_array[idx]
        joblib.dum(img, f'./train_images/{img_id}.pkl')

In [ ]:
file_name = os.path.join(data_dir, files_train[0])

file_name

In [ ]:
import time

In [ ]:
df_train = pd.read_parquet(file_name)

In [ ]:
df_train.head() # 1개의 row = 1개의 image

# row별로 읽고 쓰는 방식으로 코드를 작성하는게 효율적.
# 따라서 row별로 따로 떼서 저장하는 작업이 필요.

In [ ]:
img_ids = df_train['image_id'].values
img_array = df_train.iloc[:, 1:].values # pandas보다 numpy array로 바꿀 경우 속도가 훨씬 빨라짐.

In [ ]:
img_ids

In [ ]:
img_array.shape

In [ ]:
for idx in tqdm(range(len(df_train))):
    break

In [ ]:
img_id = img_ids[idx]
img = img_array[idx]

In [ ]:
print(img_id) # image name
print(img) # image

In [ ]:
!mkdir ./train_images

In [ ]:
# 저장하는 방법
joblib.dump(img, f'./train_images/{img_id}.pkl') # 피클이나 넘파이로 저장하면 빨리 읽어올 수 있음

In [ ]:
# 저장된 것을 불러오기

img0 = joblib.load(f'./train_images/{img_id}.pkl')

In [ ]:
img0.shape # 137 * 236

### 시간 비교

In [ ]:
start = time.time() # 시작시간 저장
img0 = joblib.load(f'./train_images/{img_id}.pkl')
print("time: ", time.time() - start)

In [ ]:
start = time.time() # 시작시간 저장
img0 = df_train.iloc[0, 1:].values
print("time: ", time.time() - start)

In [ ]:
# array 형태로 만들어서 불러오면 훨씬 빠르지만, 램이 작은 경우 하나하나 읽고 쓰는 방식이 효율적.
start = time.time() # 시작시간 저장
img0 = img_array[0]
print("time: ", time.time() - start)